In [47]:
import json
import os
import sys
from collections import Counter

import networkx as nx
import pandas as pd
import spacy
from tqdm import tqdm

tqdm.pandas()

In [2]:
wiki_pages_dir = '../out/extracted_pages/'

df_wiki_pages = pd.DataFrame(columns=['title', 'text', 'category'])
for json_file in tqdm(list(os.listdir(wiki_pages_dir))):
    with open(wiki_pages_dir+json_file, 'r') as f:
        try:
            wiki_page = json.load(f)

            wiki_page_title = wiki_page['url'].split('/')[-1]
            wiki_page_text = wiki_page['text']
            wiki_page_category = wiki_page['main_category']
            
            wiki_page_dict = {}
            wiki_page_dict['title'] = wiki_page_title
            wiki_page_dict['text'] = wiki_page_text
            wiki_page_dict['category'] = wiki_page_category

            df_wiki_pages = df_wiki_pages.append(wiki_page_dict , ignore_index=True)
        except UnicodeDecodeError:
            pass

100%|██████████| 75793/75793 [07:41<00:00, 164.26it/s]


In [3]:
df_wiki_pages.head()

,title,text,category
0,Mirosław_Miller,Mirosław Miller – Dyrektor Międzynarodowego La...,Chemia
1,Chimerokształtne,"Chimerokształtne, chimery, przerazy (Chimaerif...",Biologia
2,(2855)_Bastian,(2855) Bastian (1931 TB2) – planetoida z grupy...,Astronomia
3,Cryptocephalus_celtibericus,"""Cryptocephalus celtibericus"" – gatunek chrząs...",Biologia
4,Język_maszynowy,"Język maszynowy, kod maszynowy – zestaw rozkaz...",Matematyka


In [4]:
out_file = '../out/wiki_pages.csv'
df_wiki_pages.to_csv(out_file)

In [4]:
wiki_pages_file = '../out/wiki_pages.csv'
df_wiki_pages = pd.read_csv(wiki_pages_file, index_col=0)

In [44]:
df_wiki_pages.shape

(75792, 3)

In [56]:
def add_noun_lemmas(row):
    noun_lemmas = []
    try:
        doc = nlp(row['text'])
        for token in doc:
            if token.tag_ == 'SUBST' and token.pos_ == 'NOUN':
                noun_lemmas.append(token.lemma_.lower())
        row['noun_lemmas'] = noun_lemmas
    except TypeError:
        row['noun_lemmas'] = noun_lemmas
        print(row['text'])

    return row

df_wiki_pages = df_wiki_pages.progress_apply(add_noun_lemmas, axis=1)

  6%|▌         | 4420/75792 [02:48<41:37, 28.57it/s]nan
nan
 11%|█         | 8464/75792 [05:15<35:58, 31.19it/s]nan
nan
 13%|█▎        | 9476/75792 [05:52<33:14, 33.25it/s]nan
nan
 14%|█▍        | 10453/75792 [06:24<52:23, 20.78it/s]  nan
nan
nan
 17%|█▋        | 12741/75792 [07:45<26:25, 39.77it/s]nan
nan
 23%|██▎       | 17246/75792 [10:19<19:53, 49.07it/s]nan
nan
 23%|██▎       | 17360/75792 [10:22<22:13, 43.82it/s]nan
nan
 25%|██▌       | 19265/75792 [11:30<25:01, 37.65it/s]nan
nan
 28%|██▊       | 21535/75792 [12:43<30:38, 29.50it/s]nan
nan
 44%|████▍     | 33374/75792 [19:11<27:38, 25.58it/s]nan
nan
 54%|█████▍    | 40910/75792 [23:18<21:49, 26.64it/s]nan
nan
 55%|█████▍    | 41464/75792 [23:37<33:26, 17.11it/s]nan
nan
 60%|█████▉    | 45195/75792 [25:39<12:19, 41.36it/s]nan
nan
 67%|██████▋   | 51004/75792 [29:01<11:20, 36.41it/s]nan
nan
 69%|██████▉   | 52254/75792 [29:46<11:45, 33.35it/s]nan
nan
 72%|███████▏  | 54717/75792 [31:06<12:05, 29.06it/s]nan
nan
 78%|███████▊  | 5889

In [59]:
df_wiki_pages.head()

,title,text,category,noun_lemmas
0,Mirosław_Miller,Mirosław Miller – Dyrektor Międzynarodowego La...,Chemia,"[mirosława, miller, dyrektor, laboratorium, po..."
1,Chimerokształtne,"Chimerokształtne, chimery, przerazy (Chimaerif...",Biologia,"[chimera, przeraza, chimaeriformes, rząd, ryba..."
2,(2855)_Bastian,(2855) Bastian (1931 TB2) – planetoida z grupy...,Astronomia,"[bastian, tb2, planetoida, grupa, pas, asteroi..."
3,Cryptocephalus_celtibericus,"""Cryptocephalus celtibericus"" – gatunek chrząs...",Biologia,"[cryptocephalus, celtibericus, gatunek, chrząs..."
4,Język_maszynowy,"Język maszynowy, kod maszynowy – zestaw rozkaz...",Matematyka,"[język, koda, zestaw, rozkaz, procesor, zapis,..."


In [60]:
df_wiki_pages.shape

(75792, 4)

In [57]:
out_file = '../out/wiki_pages_lemmas.csv'
df_wiki_pages.to_csv(out_file)

## Alternative

In [ ]:
df_lemma_edgelist = pd.DataFrame(columns=['source', 'target'])
for row in tqdm(df_wiki_pages.itertuples()):
    title = row[1]
    try:
        doc = nlp(row[2])
        for token in doc:
            if token.tag_ == 'SUBST' and token.pos_ == 'NOUN':
                edge_dict = {}
                token_lemma = token.lemma_
                edge_dict['source'] = title
                edge_dict['target'] = token_lemma

                df_lemma_edgelist = df_lemma_edgelist.append(edge_dict , ignore_index=True)
    except TypeError:
        print(row[2])